<a href="https://colab.research.google.com/github/rogerwzeng/e17/blob/main/13_Lecture_13_A_Star_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import numpy as np
from typing import Callable, Dict, List, Tuple, Union

In [ ]:
# create graph with adjacency list
G = {
    "S": {
        "neighbors" : [("B", 18), ("C", 12), ("D", 30)],
        "coords"    : (5.00, 5.00)
    },
    "A": {
        "neighbors" : [("B", 27), ("G", 10)],
        "coords"    : (19.40, 18.63)
    },
    "B": {
        "neighbors" : [("A", 27), ("G", 15), ("S", 18)],
        "coords"    : (17.20, 16.99)
    },
    "C": {
        "neighbors" : [("D", 8), ("G", 20), ("S", 12)],
        "coords"    : (15.92, 4.65)
    },
    "D": {
        "neighbors" : [("C", 8), ("G", 10), ("S", 30)],
        "coords"    : (20.12, 0.95)
    },
    "G": {
        "neighbors" : [("A", 10), ("B", 15), ("C", 20), ("D", 10)],
        "coords"    : (24.36, 10.01)
    },
}

In [ ]:
# Euclidean Distance Function (serves as heuristic)
def euclidean_distance(
    coords1: Tuple[float, float],
    coords2: Tuple[float, float]) -> float:
    """
    Determines the Euclidean distance between two points
    """
    x1, y1 = coords1
    x2, y2 = coords2
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# test Euclidean distance
assert euclidean_distance((0, 0), (3,4)) == 5

In [ ]:
# A* algorithm
def a_star_algorithm(
    G: Dict[str, Dict[str, Union[List[str], Tuple[float, float]]]],
    start_node: str, goal_node: str,
    heuristic_fn: Callable = lambda x, y: 0):
    """
    Finds optimal path for a Graph G using the A* algorithm

    Inputs:
        - `G`: graph represented as an adjaceny list stored with neighbors and
            coordinates (assumes structure from above)
        - `start_node`: starting node
        - `goal_node`: goal node
        - `heuristic_fn`: heuristic function; default heuristic is to return 0

    Output:
        - Optimal path determined by A* algorithm and associated cost
    """
    ### INPUT CHECKS ###
    # check if `start_node` in G
    if start_node not in G.keys():
        raise ValueError(f"Start node {start_node} not found in Graph with keys: {G.keys()}")

    # check if `goal_node` in G
    if goal_node not in G.keys():
        raise ValueError(f"Start node {goal_node} not found in Graph with keys: {G.keys()}")

    ### A* Algorithm ###
    # initialize table with path cost, heuristic value, and parent node
    # default path cost is infinity
    # initialize heuristic value with `heuristic_fn`
    # default parent node is `None`
    a_star_table = {n: {
        "path_cost"             : np.inf,
        "heuristic"             : heuristic_fn(
            G[n]["coords"],
            G[goal_node]["coords"]),
        "parent_node"           : None,
    } for n in G.keys()}

    # update start node path cost to zero
    a_star_table[start_node]["path_cost"] = 0

    # initialize open set with starting node
    open = [(start_node, a_star_table[start_node]["path_cost"] + a_star_table[start_node]["heuristic"])]

    # initialize empty closed set
    close = []

    # iterate until open set is empty
    while len(open) != 0:
        # get curr element and cost
        curr, curr_cost = open.pop(0)
        # add to the close list
        close.append(curr)

        if curr == goal_node:
            # found goal node; recursively form the path
            path = []
            parent_node = goal_node
            while parent_node != None:
                path.append(parent_node)
                parent_node = a_star_table[parent_node]["parent_node"]

            return path[::-1], a_star_table[curr]["path_cost"]
        else:
            # check the neighbors and update table
            for neighbor in G[curr]["neighbors"]:
                # calculate the tentative_path_cost
                neighbor_node, neighor_cost = neighbor
                tentative_path_cost = a_star_table[curr]["path_cost"] + neighor_cost

                # update if tentative_path_cost is smaller than current path cost
                if tentative_path_cost < a_star_table[neighbor_node]["path_cost"]:
                    a_star_table[neighbor_node]["path_cost"] = tentative_path_cost
                    a_star_table[neighbor_node]["parent_node"] = curr

                    # add to open set - two case, new element or already present
                    nodes_in_open = [n for n, _ in open]
                    node_to_add = (neighbor_node,
                                   a_star_table[neighbor_node]["path_cost"] +
                                   a_star_table[neighbor_node]["heuristic"])
                    if neighbor in nodes_in_open:
                        open[nodes_in_open.index(neighbor)] = node_to_add
                    else:
                        open.append(node_to_add)

                    # sort open list
                    open = sorted(open, key = lambda x: x[1])


    print("Couldn't find path using A* star algorithm")
    return False

a_star_algorithm(G, "S", "G", euclidean_distance)

(['S', 'C', 'D', 'G'], 30)